In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [13]:
import pyarrow as pa
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from source import data_import

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler

In [15]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model

import keras 
from keras.models import Sequential # intitialize the ANN
from keras.layers import Dense, Activation, Dropout 

In [16]:
data = np.load('data/feature_extract_reshaped_all.npy')
data.shape

(439, 30, 71)

In [17]:
# Split data into train and test set

train_split_ratio = int((data.shape[0])*0.8)  # the first dimensio is epoch, and we are splitting the epochs between train anf test
train_data = data[:train_split_ratio,:,:]
test_data = data[train_split_ratio:,:,:]
train_data.shape, test_data.shape

((351, 30, 71), (88, 30, 71))

In [18]:
target =train_data[:,:,-1]
target.shape

(351, 30)

In [19]:
features =train_data[:,:,:-1]
features.shape


(351, 30, 70)

In [20]:
X_train = train_data[:,:,:-1]
y_train = train_data[:,:,-1]
X_test = test_data[:,:,:-1]
y_test = test_data[:,:,-1]
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")


X_train shape: (351, 30, 70)
y_train shape: (351, 30)
X_test shape: (88, 30, 70)
y_test shape: (88, 30)


In [24]:
y_train_1d = y_train.max(axis=1)
y_test_1d = y_test.max(axis=1)
print(f"y_train_1d shape: {y_train_1d.shape}")
print(f"y_test_1d shape: {y_test_1d.shape}")

y_train_1d shape: (351,)
y_test_1d shape: (88,)


In [25]:
positions = np.isnan(y_train)
np.any(positions)

False

In [66]:
X_train.shape

(351, 30, 70)

In [68]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Input
from keras.initializers import GlorotUniform

initializer = GlorotUniform()  # You can choose different initializers

model = Sequential()

# model.add(Input(shape=X_train.shape[1:]), kernel_initializer=initializer)
model.add(LSTM(units=50, input_shape=X_train.shape[1:], kernel_initializer=initializer, return_sequences=False, 
               dropout=0.1, recurrent_dropout=0.1))

# # Recurrent layer
# model.add(LSTM(64, return_sequences=False, 
#                dropout=0.1, recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.2))

# Output layer
# model.add(Dense(1, activation='sigmoid'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))
# model.add(Dense(1, activation='softmax'))


# Compile the model
from tensorflow.keras.optimizers import Adam

learning_rate = 0.01
adam_optimizer = Adam(learning_rate=learning_rate, clipvalue=2.0)

model.compile(
    optimizer=adam_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=100)
# history = model.fit(X_train, y_train_1d, epochs=10)

Epoch 1/100
11/11 [==============================] - 2s 21ms/step - loss: 0.3150 - accuracy: 0.0456
Epoch 2/100
11/11 [==============================] - 0s 23ms/step - loss: 0.1866 - accuracy: 0.0114
Epoch 3/100
11/11 [==============================] - 0s 19ms/step - loss: 0.1705 - accuracy: 0.0114
Epoch 4/100
11/11 [==============================] - 0s 21ms/step - loss: 0.1658 - accuracy: 0.0057
Epoch 5/100
11/11 [==============================] - 0s 21ms/step - loss: 0.1674 - accuracy: 0.0028
Epoch 6/100
11/11 [==============================] - 0s 21ms/step - loss: 0.1630 - accuracy: 0.0085
Epoch 7/100
11/11 [==============================] - 0s 14ms/step - loss: 0.1650 - accuracy: 0.0114
Epoch 8/100
11/11 [==============================] - 0s 10ms/step - loss: 0.1611 - accuracy: 0.0057
Epoch 9/100
11/11 [==============================] - 0s 10ms/step - loss: 0.1620 - accuracy: 0.0171
Epoch 10/100
11/11 [==============================] - 0s 10ms/step - loss: 0.1628 - accuracy: 0.0085

In [55]:
y_pred = model.predict(X_test)
y_pred

3/3 [==============================] - 0s 6ms/step


array([[1.7863083e-04, 1.3197993e-04, 1.8590198e-04, ..., 1.3677372e-01,
        1.4174931e-01, 1.2889439e-01],
       [7.9300014e-05, 6.6943401e-05, 7.6219192e-05, ..., 8.2118884e-02,
        8.4770575e-02, 7.7015303e-02],
       [1.9917441e-04, 1.4472184e-04, 2.0759609e-04, ..., 1.4762381e-01,
        1.5291797e-01, 1.3967624e-01],
       ...,
       [2.5302777e-04, 1.7230930e-04, 2.6828915e-04, ..., 1.9311816e-01,
        2.0029886e-01, 1.8665610e-01],
       [6.7393562e-06, 7.2500316e-06, 8.1746757e-06, ..., 4.8165027e-02,
        4.6492904e-02, 5.3448688e-02],
       [3.7300380e-04, 2.0142183e-04, 4.1773627e-04, ..., 4.6721911e-01,
        4.5345420e-01, 4.6927902e-01]], dtype=float32)

In [56]:
from sklearn.metrics import precision_score, recall_score
y_pred_c = y_pred.round().astype(int)
precision = precision_score(y_test, y_pred_c, average='macro')
recall = recall_score(y_test, y_pred_c, average='macro')

loss, accuracy = model.evaluate(X_test, y_test)

print('Loss:', loss)
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)

/home/weasel/reps/ai-seizure-detectives/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/weasel/reps/ai-seizure-detectives/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


3/3 [==============================] - 0s 7ms/step - loss: 0.1839 - accuracy: 0.0000e+00
Loss: 0.1839415580034256
Accuracy: 0.0
Precision: 0.145
Recall: 0.07


In [ ]:
# from keras.callbacks import EarlyStopping, ModelCheckpoint

# # Create callbacks
# callbacks = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# def make_model(input_shape):
#     input_layer = keras.layers.Input(input_shape)

#     conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
#     conv1 = keras.layers.BatchNormalization()(conv1)
#     conv1 = keras.layers.ReLU()(conv1)

#     conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
#     conv2 = keras.layers.BatchNormalization()(conv2)
#     conv2 = keras.layers.ReLU()(conv2)

#     conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
#     conv3 = keras.layers.BatchNormalization()(conv3)
#     conv3 = keras.layers.ReLU()(conv3)

#     gap = keras.layers.GlobalAveragePooling1D()(conv3)

#     output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

#     return keras.models.Model(inputs=input_layer, outputs=output_layer)


In [ ]:
# model = make_model(input_shape=X_train.shape[1:])
# keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# epochs = 500
# batch_size = 32

# callbacks = [
#     keras.callbacks.ModelCheckpoint(
#         "best_model.h5", save_best_only=True, monitor="val_loss"
#     ),
#     keras.callbacks.ReduceLROnPlateau(
#         monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
#     ),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
# ]
# model.compile(
#     optimizer="adam",
#     loss="sparse_categorical_crossentropy",
#     metrics=["sparse_categorical_accuracy"],
# )
# history = model.fit(
#     X_train,
#     y_train,
#     batch_size=batch_size,
#     epochs=epochs,
#     callbacks=callbacks,
#     #validation_split=0.2,
#     verbose=1,


inputs: A 3D tensor with shape [batch, timesteps, feature].

In [ ]:
# # Define a sequential model
# model = keras.Sequential()

# # Add an input layer with an input shape of (time_steps, input_features)
# model.add(layers.InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))

# # Add a SimpleRNN layer with 32 units (neurons)
# model.add(layers.SimpleRNN(32, activation='tanh'))

# # Add the output layer with 1 neuron and sigmoid activation for binary classification
# model.add(layers.Dense(30, activation='sigmoid'))

# # Compile the model with binary cross-entropy loss for binary classification
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Assuming X_train and y_train are your training data
# model.fit(X_train, y_train, epochs=100, batch_size=30)